In [1]:
import pandas as pd
import numpy as np
import requests
import geopandas as gpd

In [2]:
pothole = pd.read_csv("https://data.cityofchicago.org/resource/7as2-ds3y.csv?$limit=1000000")

In [3]:
patched = pd.read_csv("https://data.cityofchicago.org/resource/wqdh-9gek.csv?$limit=1000000")

In [4]:
pothole.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560478 entries, 0 to 560477
Data columns (total 19 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   creation_date                       560478 non-null  object 
 1   status                              560478 non-null  object 
 2   completion_date                     553130 non-null  object 
 3   service_request_number              560478 non-null  object 
 4   type_of_service_request             560478 non-null  object 
 5   current_activity                    393609 non-null  object 
 6   most_recent_action                  391992 non-null  object 
 7   number_of_potholes_filled_on_block  399143 non-null  float64
 8   street_address                      560476 non-null  object 
 9   zip                                 558160 non-null  float64
 10  x_coordinate                        560004 non-null  float64
 11  y_coordinate              

In [5]:
pothole.shape

(560478, 19)

In [6]:
pothole.columns

Index(['creation_date', 'status', 'completion_date', 'service_request_number',
       'type_of_service_request', 'current_activity', 'most_recent_action',
       'number_of_potholes_filled_on_block', 'street_address', 'zip',
       'x_coordinate', 'y_coordinate', 'ward', 'police_district',
       'community_area', 'ssa', 'latitude', 'longitude', 'location'],
      dtype='object')

In [7]:
patched.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179654 entries, 0 to 179653
Data columns (total 7 columns):
 #   Column                              Non-Null Count   Dtype  
---  ------                              --------------   -----  
 0   address                             179654 non-null  object 
 1   request_date                        179654 non-null  object 
 2   completion_date                     179654 non-null  object 
 3   number_of_potholes_filled_on_block  179654 non-null  int64  
 4   latitude                            179185 non-null  float64
 5   longitude                           179185 non-null  float64
 6   location                            179185 non-null  object 
dtypes: float64(2), int64(1), object(4)
memory usage: 9.6+ MB


In [8]:
patched.shape

(179654, 7)

In [9]:
patched.columns

Index(['address', 'request_date', 'completion_date',
       'number_of_potholes_filled_on_block', 'latitude', 'longitude',
       'location'],
      dtype='object')

In [23]:
pothole.sample(1)

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
520693,2011-04-14T00:00:00.000,Completed,2011-05-25T00:00:00.000,11-00827065,Pot Hole in Street,Dispatch Crew,Complete Upon Arrival,0.0,6359 S CALIFORNIA AVE,60629.0,1.158805e+06,1.862347e+06,15.0,8.0,66.0,3.0,41.777354,-87.6932,"(41.777353897793, -87.693200146949)"


In [34]:
potholes_completed = pothole[pothole['status'] == 'Completed']

In [35]:
potholes_completed

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
42,2018-12-17T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03378095,Pothole in Street,Final Outcome,Pothole Patched,7.0,600 W LAKE ST,60661.0,1.172289e+06,1.901697e+06,42.0,12.0,28.0,NaN,41.885815,-87.642847,"(41.885814724037, -87.6428471339)"
58,2018-12-17T00:00:00.000,Completed,2018-12-18T00:00:00.000,18-03378491,Pothole in Street,Final Outcome,Completed Upon Arrival,0.0,719 N DRAKE AVE,60624.0,1.152585e+06,1.904606e+06,27.0,11.0,23.0,63.0,41.894335,-87.714912,"(41.894335117217, -87.714912418158)"
236,2018-12-15T00:00:00.000,Completed,2018-12-17T00:00:00.000,18-03366015,Pothole in Street,Final Outcome,Pothole Patched,4.0,230 N OGDEN AVE,60607.0,1.166961e+06,1.901783e+06,27.0,12.0,28.0,NaN,41.885849,-87.662604,"(41.885849434411, -87.662603973878)"
263,2018-12-15T00:00:00.000,Completed,2018-12-31T00:00:00.000,18-03365265,Pothole in Street,Final Outcome,Pothole Patched,17.0,8200 S SPRINGFIELD AVE,60652.0,1.151820e+06,1.849922e+06,18.0,8.0,70.0,NaN,41.744008,-87.719444,"(41.744007628888, -87.719443672392)"
289,2018-12-14T00:00:00.000,Completed,2018-12-14T00:00:00.000,18-03360709,Pothole in Street,Final Outcome,Pothole Patched,10.0,2844 W GRACE ST,60618.0,1.156639e+06,1.925124e+06,33.0,17.0,16.0,NaN,41.950429,-87.699970,"(41.950429023486, -87.6999704208)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560473,2010-03-11T00:00:00.000,Completed,2011-03-28T00:00:00.000,10-00317212,Pot Hole in Street,Dispatch Crew,Pothole Patched,5.0,7201 W BELMONT AVE,60634.0,NaN,NaN,36.0,25.0,17.0,NaN,41.937886,-87.806806,"(41.937885966284, -87.8068055064)"
560474,2010-02-26T00:00:00.000,Completed,2011-10-04T00:00:00.000,10-00254019,Pot Hole in Street,NaN,NaN,NaN,1440 S ROCKWELL ST,60608.0,1.159269e+06,1.892879e+06,28.0,10.0,29.0,NaN,41.861757,-87.690973,"(41.861756644625, -87.690972506101)"
560475,2009-08-27T00:00:00.000,Completed,2011-01-25T00:00:00.000,09-01501980,Pot Hole in Street,Site Survey,No Problem Found,80.0,5518 N WAYNE AVE,60640.0,NaN,NaN,48.0,20.0,77.0,NaN,41.982340,-87.663869,"(41.98234020574, -87.663868977471)"
560476,2009-06-17T00:00:00.000,Completed,2011-06-07T00:00:00.000,09-01026546,Pot Hole in Street,Dispatch Crew,Pothole Patched,NaN,3845 W ADDISON ST,60618.0,1.150061e+06,1.923655e+06,30.0,17.0,21.0,NaN,41.946303,-87.724348,"(41.946303283819, -87.724347984027)"


In [36]:
potholes_no_problem = pothole[pothole['most_recent_action'] == 'No Problem Found']

In [37]:
potholes_no_problem

,creation_date,status,completion_date,service_request_number,type_of_service_request,current_activity,most_recent_action,number_of_potholes_filled_on_block,street_address,zip,x_coordinate,y_coordinate,ward,police_district,community_area,ssa,latitude,longitude,location
445620,2012-08-24T00:00:00.000,Completed,2012-08-24T00:00:00.000,12-01509005,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2300 N CLARK ST,60614.0,1.173105e+06,1.915576e+06,43.0,18.0,7.0,23.0,41.923816,-87.639555,"(41.923816495383, -87.639554930075)"
445714,2012-08-23T00:00:00.000,Completed,2012-08-24T00:00:00.000,12-01502801,Pot Hole in Street,Final Outcome,No Problem Found,0.0,155 E SUPERIOR ST,60611.0,1.177419e+06,1.905398e+06,42.0,18.0,8.0,NaN,41.895642,-87.623390,"(41.895641520773, -87.623389761865)"
445760,2012-08-23T00:00:00.000,Completed,2012-08-24T00:00:00.000,12-01501882,Pot Hole in Street,Final Outcome,No Problem Found,0.0,800 W 99TH ST,60643.0,1.172728e+06,1.839207e+06,21.0,22.0,73.0,45.0,41.714323,-87.643067,"(41.714323337011, -87.643066714219)"
445792,2012-08-22T00:00:00.000,Completed,2012-08-22T00:00:00.000,12-01494080,Pot Hole in Street,Final Outcome,No Problem Found,0.0,1727 S INDIANA AVE,60616.0,1.177932e+06,1.891926e+06,2.0,1.0,33.0,NaN,41.858299,-87.622194,"(41.858298638573, -87.622193507435)"
445908,2012-08-21T00:00:00.000,Completed,2012-08-21T00:00:00.000,12-01487704,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2000 S WENTWORTH AVE,60616.0,1.175355e+06,1.890679e+06,25.0,9.0,34.0,NaN,41.855347,-87.631993,"(41.855346944832, -87.631992669955)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
451625,2012-06-15T00:00:00.000,Completed,2012-07-18T00:00:00.000,12-01090125,Pot Hole in Street,Final Outcome,No Problem Found,0.0,800 W EASTMAN ST,60642.0,1.170751e+06,1.909618e+06,32.0,18.0,8.0,NaN,41.907582,-87.648253,"(41.907581773396, -87.648252683629)"
452125,2012-06-11T00:00:00.000,Completed,2012-07-17T00:00:00.000,12-01058012,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2000 W 21ST ST,60608.0,1.163358e+06,1.890078e+06,25.0,12.0,31.0,NaN,41.854120,-87.676020,"(41.854119832467, -87.676019670842)"
452458,2012-06-07T00:00:00.000,Completed,2012-07-23T00:00:00.000,12-01040825,Pot Hole in Street,Final Outcome,No Problem Found,0.0,4301 S HOMAN AVE,60632.0,1.154397e+06,1.875926e+06,14.0,8.0,58.0,NaN,41.815293,-87.709014,"(41.815292549968, -87.709014351831)"
452978,2012-06-01T00:00:00.000,Completed,2012-07-17T00:00:00.000,12-01015272,Pot Hole in Street,Final Outcome,No Problem Found,0.0,2009 W 21ST ST,60608.0,1.163298e+06,1.890076e+06,25.0,12.0,31.0,NaN,41.853896,-87.676318,"(41.85389629901, -87.676318396234)"
